In [1]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
import torch

In [2]:
!nvidia-smi

Fri Oct 16 20:34:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install pytorch-lightning
!pip install torch-optimizer
!pip install comet-ml

     |████████████████████████████████| 542kB 2.6MB/s 
     |████████████████████████████████| 829kB 12.2MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 276kB 15.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d714506c3d52c796ad9a0c515ff7e730f4a0259b2a0d3cbc7dfeb7d01fe32dfc
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=9fb345530d8102944775d93d1320d8a2b2a87ad8082d1cac0e3d520978ed12e5
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [4]:

import numpy as np
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torch.utils.data import random_split
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import metrics
import os

In [5]:
!git clone https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition.git
!git clone https://github.com/locuslab/TCN

Cloning into 'LSTM-Human-Activity-Recognition'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
Cloning into 'TCN'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 143 (delta 0), reused 0 (delta 0), pack-reused 140
Receiving objects: 100% (143/143), 16.22 MiB | 5.74 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [6]:
INPUT_SIGNAL_TYPES = [

    "body_acc_x_",

    "body_acc_y_",

    "body_acc_z_",

    "body_gyro_x_",

    "body_gyro_y_",

    "body_gyro_z_",

    "total_acc_x_",

    "total_acc_y_",

    "total_acc_z_"

]

 

# Output classes to learn how to classify

LABELS = [

    "WALKING",

    "WALKING_UPSTAIRS",

    "WALKING_DOWNSTAIRS",

    "SITTING",

    "STANDING",

    "LAYING"

]

In [7]:
DATA_PATH = "/content/LSTM-Human-Activity-Recognition/data/"

 

!pwd && ls

os.chdir(DATA_PATH)

!pwd && ls

 

!python download_dataset.py

 

!pwd && ls

os.chdir("..")

!pwd && ls

 

DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)

/content
LSTM-Human-Activity-Recognition  sample_data  TCN
/content/LSTM-Human-Activity-Recognition/data
download_dataset.py  source.txt

Downloading...
--2020-10-16 20:35:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/x-httpd-php]
Saving to: ‘UCI HAR Dataset.zip’

UCI HAR Dataset.zip 100%[===================>]  58.17M  16.7MB/s    in 3.5s    

2020-10-16 20:35:33 (16.7 MB/s) - ‘UCI HAR Dataset.zip’ saved [60999314/60999314]


Extracting...
Extracting successfully done to /content/LSTM-Human-Activity-Recognition/data/UCI HAR Dataset.
/content/LSTM-Human-Activity-Recognition/data
 download_dataset.py   source.txt	 'UCI HAR Dataset.zip'
 __MACOSX	      'UCI HAR Dataset'
/content/LSTM-Human-Activity-Reco

In [8]:
TRAIN = "train/"

TEST = "test/"

 

 

# Load "X" (the neural network's training and testing inputs)

 

def load_X(X_signals_paths):

    X_signals = []

 

    for signal_type_path in X_signals_paths:

        file = open(signal_type_path, 'r')

        # Read dataset from disk, dealing with text files' syntax

        X_signals.append(

            [np.array(serie, dtype=np.float32) for serie in [

                row.replace('  ', ' ').strip().split(' ') for row in file

            ]]

        )

        file.close()

 

    return np.transpose(np.array(X_signals), (1, 2, 0))

 

X_train_signals_paths = [

    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES

]

X_test_signals_paths = [

    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES

]

 

X_train = load_X(X_train_signals_paths).transpose(0,2,1)

X_test = load_X(X_test_signals_paths).transpose(0,2,1)

 

 

# Load "y" (the neural network's training and testing outputs)

 

def load_y(y_path):

    file = open(y_path, 'r')

    # Read dataset from disk, dealing with text file's syntax

    y_ = np.array(

        [elem for elem in [

            row.replace('  ', ' ').strip().split(' ') for row in file

        ]],

        dtype=np.int32

    )

    file.close()

 

    # Substract 1 to each output class for friendly 0-based indexing

    return y_ - 1

 

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"

y_test_path = DATASET_PATH + TEST + "y_test.txt"

 

y_train = (load_y(y_train_path))
y_test = (load_y(y_test_path))


In [9]:
y_train=y_train.reshape(y_train.shape[0])
y_test=y_test.reshape(y_test.shape[0])

In [10]:
scalers = {}
for i in range(X_train.shape[1]):
    scalers[i] = StandardScaler()
    X_train[:, i, :] = scalers[i].fit_transform(X_train[:, i, :]) 

for i in range(X_test.shape[1]):
    X_test[:, i, :] = scalers[i].transform(X_test[:, i, :]) 

In [11]:
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test),np.mean(X_train),np.std(X_train))

(2947, 9, 128) (2947,) -0.007139641 0.95404106 1.2971631e-10 1.0000004


In [12]:
class data_loader(Dataset):
    def __init__(self, samples, labels, t):
        self.samples = samples
        self.labels = labels
        self.T = t

    def __getitem__(self, index):
        sample, target = self.samples[index], self.labels[index]
        if self.T:
            return self.T(sample), target
        else:
            return sample, target

    def __len__(self):
        return len(self.samples)

In [13]:
batch_size = 128
def dataset_load(batch_size=batch_size):
    transform = None
    train_set = data_loader(X_train, y_train, transform)
    test_set = data_loader(X_test, y_test, transform)
    train_loader = DataLoader(
        train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [14]:
train_loader, test_loader = dataset_load()

In [15]:
x,y = next(iter(train_loader))

In [32]:
import os
from pytorch_lightning.loggers import CometLogger
comet_logger = CometLogger(
    api_key=api_key,
    save_dir='.',  # Optional
)

CometLogger will be initialized in online mode


In [23]:
from TCN.TCN.tcn import TemporalConvNet
import torch_optimizer as optim

class HARNet(pl.LightningModule):
    def __init__(self, input_size, output_size, num_channels, kernel_size, n_classes, dropout):
        super(HARNet, self).__init__()
        self.bn  = nn.BatchNorm1d(input_size)
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.logits = nn.Linear(output_size,n_classes)
        self.gelu  = nn.GELU()
        self.accuracy = pl.metrics.Accuracy()
    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        output = self.tcn(self.bn(x))#.transpose(1, 2)).transpose(1, 2)
        output = (self.linear(output[:,:,-1]))
        output = self.gelu(output)
        output = self.logits(output)
        return output
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = nn.CrossEntropyLoss()(preds, y.long())
        self.log('train_acc_step', self.accuracy(preds, y))
        self.log('train_loss', loss)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.CrossEntropyLoss()(y_hat, y.long())
        self.log('val_acc_step', self.accuracy(y_hat, y))
        self.log('val_loss', val_loss)
        return val_loss

    def configure_optimizers(self):
         optimizer  = optim.Ranger(self.parameters(), lr=0.01)
         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
         return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

In [27]:
model = HARNet(9,256,[64,64,128,128,256,256,256],5,6,dropout = 0.5)

In [25]:
from torchsummary import summary
summary(model.cuda(),(9,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 9, 128]              18
            Conv1d-2              [-1, 64, 132]           2,944
            Conv1d-3              [-1, 64, 132]           2,944
           Chomp1d-4              [-1, 64, 128]               0
           Chomp1d-5              [-1, 64, 128]               0
              ReLU-6              [-1, 64, 128]               0
              ReLU-7              [-1, 64, 128]               0
           Dropout-8              [-1, 64, 128]               0
           Dropout-9              [-1, 64, 128]               0
           Conv1d-10              [-1, 64, 132]          20,544
           Conv1d-11              [-1, 64, 132]          20,544
          Chomp1d-12              [-1, 64, 128]               0
          Chomp1d-13              [-1, 64, 128]               0
             ReLU-14              [-1, 

In [28]:
trainer = pl.Trainer(gpus = 1,logger = comet_logger,precision=16, max_epochs = 100)
trainer.fit(model, train_loader, test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/kglnsk/general/1baf3e4fa05d4dcfa6fb602c51b41deb


  | Name     | Type            | Params
---------------------------------------------
0 | bn       | BatchNorm1d     | 18    
1 | tcn      | TemporalConvNet | 2 M   
2 | linear   | Linear          | 65 K  
3 | logits   | Linear          | 1 K   
4 | gelu     | GELU            | 0     
5 | accuracy | Accuracy        | 0     


COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/kglnsk/general/1baf3e4fa05d4dcfa6fb602c51b41deb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_acc_step [114] : (0.5078125, 1.0)
COMET INFO:     train_loss [114]     : (0.02400447241961956, 1.0975849628448486)
COMET INFO:     val_acc_step [100]   : (0.4255174696445465, 0.935188353061676)
COMET INFO:     val_loss [100]       : (0.22501780092716217, 1.0222194194793701)
COMET INFO: -----------------------------------
COMET INFO: Still uploading


1